In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from mean_crossover import cross,cross_normal
import warnings
warnings.filterwarnings("ignore")

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

ativo = 'EURUSD'
#ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M15, 0, 15000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2022-06-09 01:00:00,1.07154,1.07174,1.07148,1.07171,0
1,2022-06-09 01:15:00,1.07172,1.07202,1.07168,1.07176,0
2,2022-06-09 01:30:00,1.07176,1.07191,1.07171,1.07183,0
3,2022-06-09 01:45:00,1.07185,1.07189,1.07174,1.07185,0
4,2022-06-09 02:00:00,1.07185,1.07196,1.07174,1.07179,0
...,...,...,...,...,...,...
14995,2023-01-13 22:45:00,1.08323,1.08358,1.08313,1.08325,0
14996,2023-01-13 23:00:00,1.08329,1.08335,1.08301,1.08320,0
14997,2023-01-13 23:15:00,1.08320,1.08345,1.08307,1.08335,0
14998,2023-01-13 23:30:00,1.08332,1.08333,1.08285,1.08303,0


In [3]:
pontos = 0.0005
rate_stop = 1
rate_tp = 2
timeframe = 5
ema1 = 12
ema2 = 50

for ema1 in [6,9,12]:
    for ema2 in [21,30,50]:
        for pontos in [0.0005,0.0007,0.0010, 0.0015]:
            df_acao = cross_normal(df,ema1=ema1,ema2=ema2,pontos = pontos,rate_stop = 1,rate_tp = 2,timeframe = timeframe)

            df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
            df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])

            print(f'EMA_1: {ema1} -- EMA_2: {ema2} -- pontos: {pontos} -- Timeframe: {timeframe}')

            df_new_hora = df_acao[['Hora_h','resultado_binario']]
            df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

            #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
            print(df_new_hora)

EMA_1: 6 -- EMA_2: 21 -- pontos: 0.0005 -- Timeframe: 5
   Hora_h  qtd_operacoes  taxa_acerto
0      00             11     0.272727
1      01             10     0.500000
2      02             19     0.315789
3      03             13     0.153846
4      04             11     0.272727
5      05             18     0.166667
6      06             13     0.307692
7      07             16     0.187500
8      08             22     0.454545
9      09             21     0.333333
10     10             34     0.264706
11     11             17     0.294118
12     12             23     0.391304
13     13             10     0.400000
14     14             17     0.352941
15     15             25     0.160000
16     16             24     0.291667
17     17             24     0.375000
18     18             22     0.318182
19     19             12     0.250000
20     20             14     0.428571
21     21             10     0.300000
22     22             17     0.294118
23     23              8     0.1

In [388]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]

0.32716927453769556

In [308]:
df_acao[df_acao['acao'] == 'call']['resultado_binario'].sum()/df_acao[df_acao['acao'] == 'call'].shape[0]

0.4779582366589327

In [63]:
df_new = df_acao[(df_acao['Hora_h'] >= 10) & (df_acao['Hora_h'] <= 22)]
df_new['resultado_binario'].sum()/df_new.shape[0]

KeyError: 'Hora_h'

In [26]:
df[df['cruzar_media'] != 0]

,Data,Open,High,Low,Close,Vol,EMA_12,EMA_21,diff_media,cruzar_media
36,2022-12-15 21:10:00,1.06247,1.06277,1.06238,1.06267,0,1.061985,1.061899,1,call
58,2022-12-15 23:00:00,1.06250,1.06263,1.06229,1.06233,0,1.062589,1.062591,-1,sell
68,2022-12-15 23:50:00,1.06268,1.06278,1.06266,1.06276,0,1.062516,1.062509,1,call
70,2022-12-16 00:00:00,1.06208,1.06269,1.06205,1.06223,0,1.062455,1.062490,-1,sell
76,2022-12-16 00:30:00,1.06268,1.06291,1.06268,1.06272,0,1.062494,1.062481,1,call
...,...,...,...,...,...,...,...,...,...,...
4861,2023-01-11 15:15:00,1.07471,1.07477,1.07428,1.07462,0,1.074195,1.074188,1,call
4897,2023-01-11 18:15:00,1.07577,1.07586,1.07512,1.07512,0,1.075897,1.075935,-1,sell
4927,2023-01-11 20:45:00,1.07522,1.07558,1.07511,1.07541,0,1.074972,1.074948,1,call
4967,2023-01-12 00:05:00,1.07547,1.07547,1.07514,1.07514,0,1.075525,1.075541,-1,sell
